In [3]:
# use GLCM from skimage.feature
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from skimage import data
from skimage import io
import os
# PCA from sklearn
from sklearn.decomposition import PCA
# train test split from sklearn
from sklearn.model_selection import train_test_split
# lazy predict
from lazypredict.Supervised import LazyClassifier
# brightness correction
from skimage import exposure
# HOG
from skimage.feature import hog, local_binary_pattern
# resize
from skimage.transform import resize


In [4]:
def get_patches(image, n = 10):
    # width and height of the image
    w, h = image.shape
    # patch size = w/n, h/n
    patches = []
    for i in range(n):
        for j in range(n):
            patch = image[int(i*w/n):int((i+1)*w/n), int(j*h/n):int((j+1)*h/n)]
            patches.append(patch)
    return patches

In [30]:
def glcm_properties(image, distances, angles, properties, n = 8):
    w, h = image.shape

    # make the image intensity to be integer from 0 to 255 instead of float from 0 to 1
    image_int = (image*255).astype('uint8')

    patches = get_patches(image_int, n)
    # calculate the GLCM for each patch
    glcm = [greycomatrix(patch, distances, angles, normed=True, symmetric=True) for patch in patches]

    # calculate the properties for each patch
    property_values = [np.hstack([greycoprops(g, prop).ravel() for prop in properties]) for g in glcm]

    # make property_values a numpy array
    property_values = np.array(property_values)

    # flatten the array
    property_values = property_values.flatten()

    return property_values


In [29]:
def get_features(image):
    # brightness correction
    image_corr = exposure.equalize_adapthist(image, clip_limit=0.03)

    # scale image to 400x300
    image_corr = resize(image_corr, (320, 400), anti_aliasing=True)

    # HOG
    fd= hog(image_corr, orientations=8, pixels_per_cell=(40, 50),
                        cells_per_block=(1, 1), multichannel=False)

    # GLCM
    distances = [1, 3]
    angles = [0, np.pi/4, np.pi/2]
    properties = ['energy', 'homogeneity', 'contrast']

    glcm = glcm_properties(image_corr, distances, angles, properties, n = 8)

    # concatenate HOG and GLCM
    features = np.concatenate((fd, glcm))

    return features



In [28]:
features_flattened_2 = []
label_2 = []

In [31]:

for i in range(0, 461):
    image = io.imread('flooded/'+str(i)+'.jpg', as_gray=True)
    features = get_features(image)
    features_flattened_2.append(features)
    label_2.append(1)


In [32]:
for i in range(578, 1039):
    image = io.imread('non-flooded/'+str(i)+'.jpg', as_gray=True)
    features = get_features(image)
    features_flattened_2.append(features)
    label_2.append(0)

In [33]:
features_flattened_2_np = np.array(features_flattened_2)
print(features_flattened_2_np.shape)

(922, 1632)


In [36]:
features_np = np.reshape(features_flattened_2_np, (len(features_flattened_2), 1632))
print(features_np.shape)

(922, 1632)


In [37]:
label_2_np = np.array(label_2)
print(label_2_np.shape)

(922,)


In [38]:
# shuffle the features and labels
indices = np.arange(len(label_2))
np.random.shuffle(indices)
features_np_shuffled = features_np[indices]
label_np_shuffled = label_2_np[indices]

In [39]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_np_shuffled, label_np_shuffled, test_size=0.2, random_state=42)

In [40]:
# lazy predict classifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:37<00:00,  1.31s/it]


In [41]:
models.head(20)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.89,0.89,0.89,0.89,2.32
ExtraTreesClassifier,0.89,0.89,0.89,0.89,0.57
XGBClassifier,0.87,0.87,0.87,0.87,3.35
LGBMClassifier,0.87,0.87,0.87,0.87,5.67
SVC,0.86,0.87,0.87,0.86,0.37
NuSVC,0.85,0.85,0.85,0.85,0.51
AdaBoostClassifier,0.85,0.85,0.85,0.85,9.18
CalibratedClassifierCV,0.83,0.83,0.83,0.83,2.03
LogisticRegression,0.83,0.83,0.83,0.83,0.21
